# Occupations, Salaries by State and Likelihood of Automation

## Dataset:
We are going to use following datasets:


1.   [Salaries, median hourly/annual wages broke down by occupation, provided by BLS May 2021.](https://www.bls.gov/oes/current/oes_nat.htm#11-0000)
2.   [Risk of automation broken down by occupation, provided by Carl Benedikt Frey and Michael A. Osborne](https://data.world/wnedds/occupations-by-state-and-likelihood-of-automation)
3. [Education and training assignments by detailed occupation](https://www.bls.gov/emp/tables/education-and-training-by-occupation.htm)
4. [Educational attainment for workers 25 years and older by detailed occupation](https://www.bls.gov/emp/tables/educational-attainment.htm)



## Libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

from textwrap import wrap

## Preprocessing the Dataset

Set the path here

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data1_path = '/content/drive/MyDrive/FYP/OAS/national_M2021_dl.xlsx'
data2_path = '/content/drive/MyDrive/FYP/OAS/raw_state_automation_data.csv'
data3_path = '/content/drive/MyDrive/FYP/OAS/education.xlsx'

### Importing Salary Dataset

I list the ones that are not obvious below:

* **OCC_GROUP** - The category of the size of the group. Total > Major > Minor > Broad > Detailed
* **EMP_PRSE** - Employment Relative Standard Error (i.e. how accurate the information is with higher numbers being bad)
* **H_MEAN** - Mean hourly wage
* **A_MEAN** - Mean annual wage
* **MEAN_PRSE** - Mean wage Relative Standard Error
* **H_PCT_10** - Hourly wage 10 percentile
* **H_PCT_25** - Hourly wage 25 percentile
* etc..

In [ ]:
df1 = pd.read_excel(data1_path, sheet_name='national_M2021_dl')

In [ ]:
df1.columns

Index(['AREA', 'AREA_TITLE', 'AREA_TYPE', 'PRIM_STATE', 'NAICS', 'NAICS_TITLE',
       'I_GROUP', 'OWN_CODE', 'OCC_CODE', 'OCC_TITLE', 'O_GROUP', 'TOT_EMP',
       'EMP_PRSE', 'JOBS_1000', 'LOC_QUOTIENT', 'PCT_TOTAL', 'PCT_RPT',
       'H_MEAN', 'A_MEAN', 'MEAN_PRSE', 'H_PCT10', 'H_PCT25', 'H_MEDIAN',
       'H_PCT75', 'H_PCT90', 'A_PCT10', 'A_PCT25', 'A_MEDIAN', 'A_PCT75',
       'A_PCT90', 'ANNUAL', 'HOURLY'],
      dtype='object')

In [ ]:
df1.head()

,AREA,AREA_TITLE,AREA_TYPE,PRIM_STATE,NAICS,NAICS_TITLE,I_GROUP,OWN_CODE,OCC_CODE,OCC_TITLE,...,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
0,99,U.S.,1,US,0,Cross-industry,cross-industry,1235,00-0000,All Occupations,...,22,32.98,49.43,23980,29950,45760,68590,102810,NaN,NaN
1,99,U.S.,1,US,0,Cross-industry,cross-industry,1235,11-0000,Management Occupations,...,49.25,77.39,#,47860,74710,102450,160960,#,NaN,NaN
2,99,U.S.,1,US,0,Cross-industry,cross-industry,1235,11-1000,Top Executives,...,47.46,77.18,#,41260,60900,98720,160540,#,NaN,NaN
3,99,U.S.,1,US,0,Cross-industry,cross-industry,1235,11-1010,Chief Executives,...,86.31,#,#,60300,111080,179520,#,#,NaN,NaN
4,99,U.S.,1,US,0,Cross-industry,cross-industry,1235,11-1011,Chief Executives,...,86.31,#,#,60300,111080,179520,#,#,NaN,NaN


### Importing Automation Dataset

**Note:** Perhaps most importantly, the Probability column in automation_data represents the probability of the occupation being automated with a probability greater than 0.7 representing a **`high risk category, meaning that associated occupations are potentially automatable over some unspecified number of years, perhaps a decade or two`**

In [ ]:
df2 = pd.read_csv(data2_path, encoding = "ISO-8859-1")
states = df2.columns[3:]

In [ ]:
len(states)

51

In [ ]:
df2.head()

,SOC,Occupation,Probability,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,11-1011,Chief Executives,0.015,1030,760,5750,2710,31150,880,1410,...,560,5460,5890,3650,280,6320,5910,980,3740,160
1,11-1021,General and Operations Managers,0.160,26930,6490,43300,20680,261780,41540,33280,...,3730,44400,168610,36200,2760,52380,43760,10200,32350,4840
2,11-2011,Advertising and Promotions Managers,0.039,50,40,470,110,3760,480,300,...,0,670,1210,380,40,240,640,40,200,0
3,11-2021,Marketing Managers,0.014,530,200,4790,1090,33390,3060,4970,...,60,3400,9570,2320,380,3840,5830,260,2980,30
4,11-2022,Sales Managers,0.013,2510,400,10650,2650,69180,4570,7040,...,300,8890,22310,3360,480,5360,7390,600,5730,180


### Importing education data

In [ ]:
df3 = pd.read_excel(data3_path, sheet_name='Table 5.3', skiprows=1)
df3.head()

,2021 National Employment Matrix title,2021 National Employment Matrix code,Less than high school diploma,High school diploma or equivalent,"Some college, no degree",Associate's degree,Bachelor's degree,Master's degree,Doctoral or professional degree
0,"Total, all occupations",00-0000,8.0,23.6,20.2,9.6,23.7,10.6,4.4
1,Chief executives(1),11-1011,1.5,8.3,14.2,5.1,40.5,23.7,6.7
2,General and operations managers,11-1021,2.6,17.0,24.8,9.8,33.0,11.1,1.6
3,Legislators(1),11-1031,1.5,8.3,14.2,5.1,40.5,23.7,6.7
4,Advertising and promotions managers,11-2011,0.8,3.9,10.8,6.0,63.6,12.8,2.1


In [ ]:
education1 = df3.drop(df3.columns[1], axis=1)
education1 = education1.rename(columns={"2021 National Employment Matrix title": "occupation",
                                       "Less than high school diploma": "lessthanhs",
                                       "High school diploma or equivalent": "hsdiploma",
                                       "Some college, no degree": "somecollege",
                                       "Associate's degree": "associates",
                                       "Bachelor's degree": "bachelors",
                                       "Master's degree": "masters",
                                       "Doctoral or professional degree": "professional"})

education2 = education1.groupby('occupation').agg({
    'lessthanhs': 'sum',
    'hsdiploma': 'sum',
    'somecollege': 'sum',
    'associates': 'sum',
    'bachelors': 'sum',
    'masters': 'sum',
    'professional': 'sum'
}).reset_index()

education2['hsorless'] = education2['lessthanhs'] + education2['hsdiploma']
education2['somecollegeorassociates'] = education2['somecollege'] + education2['associates']
education2['postgrad'] = education2['masters'] + education2['professional']

education_data1 = education2.dropna().drop(0)
education_data1['occupation'] = education_data1['occupation'].str.lower()


In [ ]:
education_data1.head()

,occupation,lessthanhs,hsdiploma,somecollege,associates,bachelors,masters,professional,hsorless,somecollegeorassociates,postgrad
1,accountants and auditors,0.0,3.0,5.5,8.0,57.0,23.4,3.0,3.0,13.5,26.4
2,actors,2.1,14.0,16.5,5.4,47.9,13.2,0.9,16.1,21.9,14.1
3,actuaries,0.0,0.0,2.0,0.4,62.2,23.8,11.7,0.0,2.4,35.5
4,acupuncturists,1.1,1.4,2.6,1.2,13.3,51.0,29.5,2.5,3.8,80.5
5,adhesive bonding machine operators and tenders,18.7,43.3,21.8,12.8,1.5,1.9,0.0,62.0,34.6,1.9


In [ ]:
df4 = pd.read_excel(data3_path, sheet_name='Table 5.4', skiprows=1)

df4 = df4.rename(columns={"2021 National Employment Matrix title": "occupation",
                                       "Typical education needed for entry": "typical",
                                       "Work experience in a related occupation": "workexp"})

typicaleducation_data = df4[['occupation', 'typical', 'workexp']]
typicaleducation_data = typicaleducation_data.dropna()
typicaleducation_data['occupation'] = typicaleducation_data['occupation'].str.lower()
# typicaleducation_data = typicaleducation_data.rename(columns={ 'occupation':'Occupation'})

In [ ]:
typicaleducation_data.head()

,occupation,typical,workexp
0,chief executives,Bachelor's degree,5 years or more
1,general and operations managers,Bachelor's degree,5 years or more
2,legislators,Bachelor's degree,Less than 5 years
3,advertising and promotions managers,Bachelor's degree,Less than 5 years
4,marketing managers,Bachelor's degree,5 years or more


### Joining the datasets

For my next step, I'm going to look at how compatible the datatables are. The tables are not from the same original source as one comes from a research paper and the other from the US Bureau of Labor Statistics. In an ideal world, I could just join the tables together matching Occupation to OCC_TITLE, but before I do I should check to see how much overlap there is.

In [ ]:
temp_occ_1 = df1['OCC_TITLE'].unique()
temp_occ_2 = df2['Occupation'].unique()
len(temp_occ_1) , len(temp_occ_2)

(1138, 702)

In [ ]:
# what I've done first is check what percent of unique occupations in the
# automation_data table are in the occupation_salary table
total_diff = len(np.setdiff1d(temp_occ_1, temp_occ_2))
percent_diff = f"{len(np.intersect1d(temp_occ_2, temp_occ_1)) / len(temp_occ_2) * 100: .1f}%"
print(f"Total different values are: {total_diff}, contributing: {percent_diff}")

Total different values are: 730, contributing:  58.1%


**Note:** something is wrong as it is showing only 58% overlapping we'll look into the fields

it turned out to primarily be a simple string substitution. Since automation_data was originally in a CSV format, the creator of the original file used semi-colons instead of the commas used in the occupation_salary table.

In [ ]:
temp_occ_2 = np.array([re.sub(';', ',', s) for s in temp_occ_2])

In [ ]:
total_diff = len(np.setdiff1d(temp_occ_1, temp_occ_2))
percent_diff = f"{len(np.intersect1d(temp_occ_2, temp_occ_1)) / len(temp_occ_2) * 100: .1f}%"
print(f"Total different values are: {total_diff}, contributing: {percent_diff}")

Total different values are: 552, contributing:  83.5%


For my next step,


1.   I needed to join the two tables together. I started by doing the string substitution on the automation_data table and by adding a **State_Total** column to that dataset, so that I could compare it to the **TOT_EMP** variable from the occupation_salary table.
2.   I wanted to do was ensure that every occupation's salary was measured annually. Looking at the dataset, it became clear that virtually all annual mean salaries were simply the hourly salary *multiplied by 2080 (40 hours a week, 52 weeks in a year)*. I took both columns and converted them to numeric variables and then replaced the missing values in **A_MEAN** with a calculated value based on **H_MEAN**
3. I joined the two tables together using an **inner join** to ensure that every occupation in the combined table has a **Mean Annual Salary, Total Employment value, a Sum of the Employment by State, and a Probability of Automation**



In [ ]:
df1.columns

Index(['AREA', 'AREA_TITLE', 'AREA_TYPE', 'PRIM_STATE', 'NAICS', 'NAICS_TITLE',
       'I_GROUP', 'OWN_CODE', 'OCC_CODE', 'OCC_TITLE', 'O_GROUP', 'TOT_EMP',
       'EMP_PRSE', 'JOBS_1000', 'LOC_QUOTIENT', 'PCT_TOTAL', 'PCT_RPT',
       'H_MEAN', 'A_MEAN', 'MEAN_PRSE', 'H_PCT10', 'H_PCT25', 'H_MEDIAN',
       'H_PCT75', 'H_PCT90', 'A_PCT10', 'A_PCT25', 'A_MEDIAN', 'A_PCT75',
       'A_PCT90', 'ANNUAL', 'HOURLY'],
      dtype='object')

In [ ]:
automation_data2 = df2.copy()
automation_data2['Occupation'] = automation_data2['Occupation'].str.replace(';', ',')
automation_data2['State_Total'] = automation_data2.iloc[:, 4:54].sum(axis=1)

occupation_salary_red = df1[['OCC_TITLE', 'O_GROUP', 'TOT_EMP', 'EMP_PRSE', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE', 'A_MEDIAN']].drop_duplicates()
occupation_salary_red['H_MEAN'] = pd.to_numeric(occupation_salary_red['H_MEAN'], errors='coerce')
occupation_salary_red['A_MEAN'] = pd.to_numeric(occupation_salary_red['A_MEAN'], errors='coerce')

occupation_salary_red.loc[occupation_salary_red['A_MEAN'].isna(), 'A_MEAN'] = occupation_salary_red.loc[occupation_salary_red['A_MEAN'].isna(), 'H_MEAN'] * 2080

occupation_salary_red = occupation_salary_red.drop(columns=['H_MEAN'])

combined_data = pd.merge(occupation_salary_red, automation_data2.drop(columns=['SOC']), how='inner', left_on='OCC_TITLE', right_on='Occupation')
combined_data = combined_data.dropna(subset=['State_Total'])
combined_data = combined_data.rename(columns={ 'Occupation':'occupation'})


In [ ]:
typicaleducation_data['occupation'] #831

0                               chief executives
1                general and operations managers
2                                    legislators
3            advertising and promotions managers
4                             marketing managers
                         ...                    
827      pump operators, except wellhead pumpers
828                             wellhead pumpers
829    refuse and recyclable material collectors
830            tank car, truck, and ship loaders
831           material moving workers, all other
Name: occupation, Length: 832, dtype: object

In [ ]:
combined_data.head() #803

,OCC_TITLE,O_GROUP,TOT_EMP,EMP_PRSE,A_MEAN,MEAN_PRSE,A_MEDIAN,occupation,Probability,Alabama,...,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming,State_Total
0,Chief Executives,broad,200480,1.6,213020.0,0.8,179520,Chief Executives,0.015,1030,...,5460,5890,3650,280,6320,5910,980,3740,160,222240
1,Chief Executives,detailed,200480,1.6,213020.0,0.8,179520,Chief Executives,0.015,1030,...,5460,5890,3650,280,6320,5910,980,3740,160,222240
2,General and Operations Managers,broad,2984920,0.5,115250.0,0.4,97970,General and Operations Managers,0.160,26930,...,44400,168610,36200,2760,52380,43760,10200,32350,4840,2161940
3,General and Operations Managers,detailed,2984920,0.5,115250.0,0.4,97970,General and Operations Managers,0.160,26930,...,44400,168610,36200,2760,52380,43760,10200,32350,4840,2161940
4,Advertising and Promotions Managers,broad,22520,3.4,142860.0,1.9,127150,Advertising and Promotions Managers,0.039,50,...,670,1210,380,40,240,640,40,200,0,28030


In [ ]:


# Replace ";" with "," in the "occupation" column and convert it to lowercase
combined_data['occupation'] = combined_data['occupation'].str.lower()

# Merge "edsal2" and "automation1" dataframes by "occupation"
combined_data_2 = pd.merge(typicaleducation_data, combined_data, on='occupation')


In [ ]:
WAPE = f"{np.mean(abs(combined_data['State_Total'] - combined_data['TOT_EMP']) / np.mean(combined_data['TOT_EMP'])) * 100: .1f}%"

In [ ]:
# print('Weighted Absolute Percentage Error (WAPE)', WAPE)

In [ ]:
combined_data.columns

Index(['OCC_TITLE', 'O_GROUP', 'TOT_EMP', 'EMP_PRSE', 'A_MEAN', 'MEAN_PRSE',
       'A_MEDIAN', 'occupation', 'Probability', 'Alabama', 'Alaska', 'Arizona',
       'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware',
       'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming', 'State_Total'],
      dtype='object')

## Analysis

### Which occupations contribute most to the American economy (in USD).

In [ ]:
temp1 = df1.groupby('OCC_TITLE').apply(lambda x: x.assign(natlwage=x['TOT_EMP'] * pd.to_numeric(x['A_MEAN'], errors='coerce')))
temp1 = temp1.dropna(subset=['TOT_EMP', 'A_MEAN', 'A_MEDIAN'])

temp1['A_MEDIAN'] = pd.to_numeric(temp1['A_MEDIAN'], errors='coerce')
temp2 = temp1[['OCC_TITLE', 'TOT_EMP', 'A_MEDIAN', 'natlwage']].drop_duplicates()

temp2 = temp2.sort_values(by='natlwage', ascending=False).reset_index(drop=True)
temp2 = temp2.rename(columns = {'OCC_TITLE' : 'occupation',
                                    'TOT_EMP' : 'Total Employees',
                                    'A_MEDIAN' : '$ Annual Median',
                                    'natlwage' : 'Net wage'})
temp2[1:11].to_csv('total_wages.csv')

<ipython-input-36-fea45379387f>:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  temp1 = df1.groupby('OCC_TITLE').apply(lambda x: x.assign(natlwage=x['TOT_EMP'] * pd.to_numeric(x['A_MEAN'], errors='coerce')))


In [ ]:
filtered_salary2 = temp2[temp2['Total Employees'] < 15000000]

fig = px.scatter(filtered_salary2, x='Total Employees', y='$ Annual Median',
                 color='$ Annual Median', opacity=0.5, color_continuous_scale='jet')
fig.update_layout(
    title="Median Wage vs Total Employment",
    xaxis_title="Number of Americans in a Given Job",
    yaxis_title="Median Wage of a Given Job",
)
fig.update_xaxes(showgrid=True, gridwidth=1)
fig.show()

In [ ]:
temp2['occupation'] = temp2['occupation'].str.lower()

edsal = pd.merge(education_data1, temp2, on='occupation')
edsal = edsal.dropna()

edsal.head()

,occupation,lessthanhs,hsdiploma,somecollege,associates,bachelors,masters,professional,hsorless,somecollegeorassociates,postgrad,Total Employees,$ Annual Median,Net wage
0,accountants and auditors,0.0,3.0,5.5,8.0,57.0,23.4,3.0,3.0,13.5,26.4,1318550,77250.0,1.107318e+11
2,actuaries,0.0,0.0,2.0,0.4,62.2,23.8,11.7,0.0,2.4,35.5,23040,105900.0,2.886912e+09
3,acupuncturists,1.1,1.4,2.6,1.2,13.3,51.0,29.5,2.5,3.8,80.5,7250,60570.0,5.203325e+08
4,adhesive bonding machine operators and tenders,18.7,43.3,21.8,12.8,1.5,1.9,0.0,62.0,34.6,1.9,12510,37630.0,4.826358e+08
5,administrative services managers,1.0,13.7,20.0,12.4,34.4,16.3,2.1,14.7,32.4,18.4,224620,100170.0,2.538880e+10


In [ ]:
typicaleducation_data.head()

,occupation,typical,workexp
0,chief executives,Bachelor's degree,5 years or more
1,general and operations managers,Bachelor's degree,5 years or more
2,legislators,Bachelor's degree,Less than 5 years
3,advertising and promotions managers,Bachelor's degree,Less than 5 years
4,marketing managers,Bachelor's degree,5 years or more


In [ ]:
typicaleducation_data.shape, edsal.shape

((832, 3), (368, 14))

In [ ]:
edsal2 = pd.merge(edsal, typicaleducation_data, on='occupation')

edsal2.head()

,occupation,lessthanhs,hsdiploma,somecollege,associates,bachelors,masters,professional,hsorless,somecollegeorassociates,postgrad,Total Employees,$ Annual Median,Net wage,typical,workexp
0,accountants and auditors,0.0,3.0,5.5,8.0,57.0,23.4,3.0,3.0,13.5,26.4,1318550,77250.0,1.107318e+11,Bachelor's degree,None
1,actuaries,0.0,0.0,2.0,0.4,62.2,23.8,11.7,0.0,2.4,35.5,23040,105900.0,2.886912e+09,Bachelor's degree,None
2,acupuncturists,1.1,1.4,2.6,1.2,13.3,51.0,29.5,2.5,3.8,80.5,7250,60570.0,5.203325e+08,Master's degree,None
3,adhesive bonding machine operators and tenders,18.7,43.3,21.8,12.8,1.5,1.9,0.0,62.0,34.6,1.9,12510,37630.0,4.826358e+08,High school diploma or equivalent,None
4,administrative services managers,1.0,13.7,20.0,12.4,34.4,16.3,2.1,14.7,32.4,18.4,224620,100170.0,2.538880e+10,Bachelor's degree,Less than 5 years


### What is the median wage for each typical level of education?

In [ ]:
edsal3 = edsal2.groupby('typical').agg({'$ Annual Median': 'mean'}).sort_values('$ Annual Median').reset_index()
edsal3

,typical,$ Annual Median
0,No formal educational credential,32859.756098
1,"Some college, no degree",41000.000000
2,Postsecondary nondegree award,43859.230769
3,High school diploma or equivalent,45767.142857
4,Associate's degree,61823.125000
5,Master's degree,82488.823529
6,Bachelor's degree,83390.736842
7,Doctoral or professional degree,97990.000000


In [ ]:
fig = px.bar(edsal3, x='typical', y='$ Annual Median', color=edsal3['typical'],
             title='Median Annual Income by Education Level',
             labels={'typical': 'Education Level', 'medianwage': 'Median Annual Income'},
        )
fig.update_xaxes(title_text=None, showticklabels=False)
fig.show()

### Visualize the probability of automation vs total employment:

In [ ]:
combined_data.columns

Index(['OCC_TITLE', 'O_GROUP', 'TOT_EMP', 'EMP_PRSE', 'A_MEAN', 'MEAN_PRSE',
       'A_MEDIAN', 'occupation', 'Probability', 'Alabama', 'Alaska', 'Arizona',
       'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware',
       'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming', 'State_Total'],
      dtype='object')

In [ ]:
combined_data_2.columns

Index(['occupation', 'typical', 'workexp', 'OCC_TITLE', 'O_GROUP', 'TOT_EMP',
       'EMP_PRSE', 'A_MEAN', 'MEAN_PRSE', 'A_MEDIAN', 'Probability', 'Alabama',
       'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
       'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
       'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan',
       'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming', 'State_Total'],
      dtype='object')

In [ ]:
combined_data_2['occupation'].drop_duplicates().to_csv('occupation.csv')

In [ ]:
fig = px.scatter(combined_data, x='State_Total', y='Probability',
                 title='Probability of Automation vs Total Employment',
                 labels={'total': 'Number of Americans in a Given Job', 'probability': 'Probability of Automation'},
                 color_discrete_sequence=['red'])
fig.update_traces(opacity=1/3)
# fig.update_xaxes(type='log')
fig.update_yaxes(range=[0,1])
fig.add_trace(px.scatter(combined_data[combined_data['State_Total'] > 4000000], x='State_Total', y='Probability',
                 labels={'total': 'Number of Americans in a Given Job', 'probability': 'Probability of Automation'},
                 text='occupation', size_max=2.5).update_traces(textposition='top right', showlegend=False).data[0])
fig.show()

There doesn’t seem to be a huge relationship between automation and number of employees

### Likelihood of Job Automation by Education Level

In [ ]:
edu_vs_auto = combined_data_2.groupby('typical').agg({'Probability': 'mean'}).sort_values('Probability').reset_index()


In [ ]:
edu_vs_auto

,typical,Probability
0,Doctoral or professional degree,0.088572
1,Master's degree,0.116591
2,Bachelor's degree,0.205874
3,Associate's degree,0.460835
4,Postsecondary nondegree award,0.486500
5,High school diploma or equivalent,0.710718
6,No formal educational credential,0.803514
7,"Some college, no degree",0.824286


In [ ]:
fig = px.bar(data_frame=edu_vs_auto, x='typical', y='Probability', color='typical',
             color_discrete_sequence=px.colors.qualitative.Pastel)

fig.update_traces(marker=dict(line=dict(color='black', width=0.5)), opacity=1)
fig.update_yaxes(title="Likelihood of Job Automation")
fig.update_layout(title="Likelihood of Job Automation by Education Level",
                  legend_title_text='Education Level',
                  showlegend=True,
                  legend=dict(itemclick=False, itemdoubleclick=False))
fig.update_xaxes(title=None, showticklabels=False)
fig.show()


### Likelihood of Job Automation vs Median Income

In [ ]:
fig = px.scatter(combined_data_2, x='A_MEAN', y='Probability', size='TOT_EMP', color='typical',
                 opacity=0.5, color_discrete_sequence=px.colors.qualitative.Vivid,
                 title='Likelihood of Job Automation vs Median Income',
                 labels={'A_MEDIAN': 'Median Income', 'probability': 'Probability of Automation'},
                 size_max=30, range_y=[-.01, 1])

fig.update_traces(marker=dict(line=dict(width=0.5, color='Black')))
fig.update_layout(showlegend=True, legend_title='Education Level', legend=dict(title_font=dict(size=10)))
fig.update_xaxes(title_font=dict(size=12))
fig.update_yaxes(title_font=dict(size=12))

fig.show()

In [ ]:
table_1 = combined_data_2[(combined_data_2['A_MEAN'] > 175000) & (combined_data_2['Probability'] < 0.05)]

In [ ]:
table_1

,occupation,typical,workexp,OCC_TITLE,O_GROUP,TOT_EMP,EMP_PRSE,A_MEAN,MEAN_PRSE,A_MEDIAN,...,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming,State_Total
0,chief executives,Bachelor's degree,5 years or more,Chief Executives,broad,200480,1.6,213020.0,0.8,179520,...,5460,5890,3650,280,6320,5910,980,3740,160,222240
1,chief executives,Bachelor's degree,5 years or more,Chief Executives,detailed,200480,1.6,213020.0,0.8,179520,...,5460,5890,3650,280,6320,5910,980,3740,160,222240
258,oral and maxillofacial surgeons,Doctoral or professional degree,None,Oral and Maxillofacial Surgeons,detailed,5330,12.3,311460.0,3.7,#,...,100,670,0,0,160,0,0,0,0,3450
259,orthodontists,Doctoral or professional degree,None,Orthodontists,detailed,5140,11.4,267280.0,6,#,...,40,360,0,0,0,0,0,0,0,3800


In [ ]:
table_1 = table_1[['occupation', 'typical', 'TOT_EMP', 'Probability']].drop_duplicates()
# table_1 = table_1.re

In [ ]:
fig = px.scatter(combined_data_2, x='A_MEAN', y='Probability', size='TOT_EMP', color='typical',
                 opacity=0.5, color_discrete_sequence=px.colors.qualitative.Vivid,
                 title='Likelihood of Job Automation vs Median Income',
                 labels={'A_MEDIAN': 'Median Income', 'probability': 'Probability of Automation'},
                 size_max=30, range_y=[-.01, 1], trendline="ols", trendline_scope="overall"
                 ,trendline_color_override="black")

fig.update_traces(marker=dict(line=dict(width=0.5, color='Black')))
fig.update_layout(showlegend=True, legend_title='Education Level', legend=dict(title_font=dict(size=10)))
fig.update_xaxes(title_font=dict(size=12))
fig.update_yaxes(title_font=dict(size=12))


fig.add_trace(px.scatter(combined_data_2[
    (combined_data_2['A_MEAN'] > 175000) & (combined_data_2['Probability'] < 0.05)], x='A_MEAN', y='Probability',
                 labels={'total': 'Number of Americans in a Given Job', 'probability': 'Probability of Automation'},
                 text='occupation', size_max=2.5).update_traces(textposition='top center', showlegend=False).data[0])
fig.add_trace(px.scatter(combined_data_2[
    (combined_data_2['A_MEAN'] > 100000) & (combined_data_2['Probability'] > 0.90)], x='A_MEAN', y='Probability',
                 labels={'total': 'Number of Americans in a Given Job', 'probability': 'Probability of Automation'},
                 text='occupation', size_max=2.5).update_traces(textposition='bottom center', showlegend=False).data[0])

fig.show()

In [ ]:
states = combined_data_2.columns[11:-1]

In [ ]:
p = combined_data_2.sort_values(by=['Probability'], ascending=False)
sum_work_per_state = p[states].sum()
States_sum_DF = pd.DataFrame({'States':sum_work_per_state.index, 'sum_work_positions':sum_work_per_state.values})
States_sum_sort =  States_sum_DF.sort_values(by=['sum_work_positions'], ascending=False)

In [ ]:
States_sum_sort.head()

,States,sum_work_positions
4,California,17701210
43,Texas,13326870
32,New York,10149740
9,Florida,9647870
13,Illinois,6447580


In [ ]:

# Define a dictionary of state abbreviations
state_abbr = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
}


In [ ]:
len(state_abbr)

51

In [ ]:

# Use the map() function to apply the state abbreviations to the DataFrame
States_sum_sort["Abbreviation"] = States_sum_sort["States"].map(state_abbr)

# Print the resulting DataFrame
States_sum_sort.head()


,States,sum_work_positions,Abbreviation
4,California,17701210,CA
43,Texas,13326870,TX
32,New York,10149740,NY
9,Florida,9647870,FL
13,Illinois,6447580,IL


### Total number of jobs positions per state

In [ ]:
# Use Plotly Express to create a choropleth plot
fig = px.choropleth(States_sum_sort,
                    locations='Abbreviation',  # Column with state names
                    locationmode='USA-states',  # Use USA states as location mode
                    color='sum_work_positions',  # Column with values to be represented by color
                    scope='usa',  # Set the scope to USA
                    color_continuous_scale='Viridis',  # Choose a color scale
                    labels={'sum_work_positions': 'Total Workers'},  # Set custom labels
                    title='Total Employees per States')  # Set a title for the plot

# Show the plot
fig.show()


In [ ]:
def barplot(data_in, x_data, y_data, title_in, hue_in, line):
    fig = px.bar(data_in, x=x_data, y=y_data, color=hue_in, barmode='group')
    fig.update_layout(title='\n'.join(title_in), title_font=dict(size=20, color='white', family='Arial'),
                      xaxis_title=title_in , yaxis_title='Value')
    fig.add_shape(type='line', x0=line, x1=line, y0=0, y1=1, xref='x', yref='paper', line=dict(color='red'))
    return fig


In [ ]:
# barplot(States_sum_sort, 'sum_work_positions', 'States', "Total number of jobs positions per state", None, 0)
barplot(States_sum_sort, 'sum_work_positions', 'States', "Total number of jobs positions per state", None, 0)

### Jobs lost per state when we use automation `thershold < 0.7`

In [ ]:
THERSHOLD = 0.7

In [ ]:
p_less_07_df = combined_data_2.loc[(combined_data_2.Probability < THERSHOLD)]

In [ ]:
sum_work_per_state_p_less_07 = p_less_07_df[states].sum()
States_sum_DF_p_less_07 = pd.DataFrame({'States':sum_work_per_state_p_less_07.index, 'sum_work_positions':sum_work_per_state_p_less_07.values})
States_sum_sort_p_less_07 =  States_sum_DF_p_less_07.sort_values(by=['sum_work_positions'], ascending=False)
States_sum_sort_p_less_07.head()

,States,sum_work_positions
4,California,8055980
43,Texas,5872910
32,New York,4771540
9,Florida,4122890
13,Illinois,3008160


In [ ]:
barplot(States_sum_sort_p_less_07, 'sum_work_positions', 'States', "Total Jobs per state when p<0.7", None, 0)

In [ ]:
p_less_07_df = combined_data_2.loc[(combined_data_2.Probability < THERSHOLD)]

In [ ]:
o2 = combined_data_2.loc[(combined_data_2.Probability < 0.2)]

o2 = o2[states].sum()
o2 = pd.DataFrame({'States':o2.index, 'sum_work_positions':o2.values})
o2 =  o2.sort_values(by=['sum_work_positions'], ascending=False)

o3 = combined_data_2.loc[(combined_data_2.Probability < 0.3)]

o3 = o3[states].sum()
o3 = pd.DataFrame({'States':o3.index, 'sum_work_positions':o3.values})
o3 =  o3.sort_values(by=['sum_work_positions'], ascending=False)

o4 = combined_data_2.loc[(combined_data_2.Probability < 0.4)]

o4 = o4[states].sum()
o4 = pd.DataFrame({'States':o4.index, 'sum_work_positions':o4.values})
o4 =  o4.sort_values(by=['sum_work_positions'], ascending=False)

o5 = combined_data_2.loc[(combined_data_2.Probability < 0.5)]

o5 = o5[states].sum()
o5 = pd.DataFrame({'States':o5.index, 'sum_work_positions':o5.values})
o5 =  o5.sort_values(by=['sum_work_positions'], ascending=False)

o6 = combined_data_2.loc[(combined_data_2.Probability < 0.6)]

o6 = o6[states].sum()
o6 = pd.DataFrame({'States':o6.index, 'sum_work_positions':o6.values})
o6 =  o6.sort_values(by=['sum_work_positions'], ascending=False)

# o7 = combined_data_2.loc[(combined_data_2.Probability < 0.7)]

# o7 = o7[states].sum()
# o7 = pd.DataFrame({'States':o7.index, 'sum_work_positions':o7.values})
# o7 =  o7.sort_values(by=['sum_work_positions'], ascending=False)

o8 = combined_data_2.loc[(combined_data_2.Probability < 0.8)]

o8 = o8[states].sum()
o8 = pd.DataFrame({'States':o8.index, 'sum_work_positions':o8.values})
o8 =  o8.sort_values(by=['sum_work_positions'], ascending=False)


o2['Threshhold'] = 0.2
o3['Threshhold'] = 0.3
o4['Threshhold'] = 0.4
o5['Threshhold'] = 0.5
o6['Threshhold'] = 0.6
# o2['Threshhold'] = 0.7
o8['Threshhold'] = 0.8

In [ ]:
States_sum_sort['Threshhold'] = 1.0
States_sum_sort_p_less_07['Threshhold'] = THERSHOLD

In [ ]:
Compare_thresholds = pd.concat([States_sum_sort, o8, States_sum_sort_p_less_07, o6, o5, o4, o3, o2])

In [ ]:
Compare_thresholds_01_07 = pd.concat([States_sum_sort, States_sum_sort_p_less_07])

In [ ]:
fig = px.bar(Compare_thresholds, x="sum_work_positions", y="States", color="States",
             animation_frame="Threshhold",
             color_discrete_sequence=px.colors.qualitative.Pastel)  # Specify color scale

# Customize the appearance of the chart
fig.update_traces(marker_line_width=0)  # Remove marker outlines
fig.update_layout(title="Comparison of Work Positions by State",  # Add title
                  xaxis_title="Sum of Work Positions",  # Add x-axis title
                  yaxis_title="States" , height=800, xaxis_range=[0 , 18000000] )  # Add y-axis title
fig.show()

In [ ]:
Compare_thresholds_01_07

,States,sum_work_positions,Abbreviation,Threshhold
4,California,17701210,CA,1.0
43,Texas,13326870,TX,1.0
32,New York,10149740,NY,1.0
9,Florida,9647870,FL,1.0
13,Illinois,6447580,IL,1.0
...,...,...,...,...
34,North Dakota,208770,NaN,0.7
41,South Dakota,182940,NaN,0.7
1,Alaska,166830,NaN,0.7
45,Vermont,143110,NaN,0.7


In [ ]:
fig = px.histogram(Compare_thresholds_01_07.sort_values(by=['sum_work_positions'], ascending=True),
             x='sum_work_positions', y='States', color='Threshhold',  barmode='group')

fig.update_layout(title='\n'.join("Total number of jobs positions per state now (Threshhold=1.0) and when higher automation probability accupations lost (Threshhold=threshhold)"),
                  title_font=dict(size=20, color='white', family='Arial'),
                  xaxis_title="Total number of jobs positions per state now (Threshhold=1.0) and when higher automation probability accupations lost (Threshhold=threshhold)" ,
                  yaxis_title='Value', height=800)
# fig.add_shape(type='line', x0=0, x1=0, y0=0, y1=1, xref='x', yref='paper', line=dict(color='red'))

### what are reliative loss numbers

In [ ]:
States_sum_join = States_sum_sort.join(States_sum_sort_p_less_07, lsuffix='', rsuffix='0.5')
States_sum_drop = States_sum_join.drop(['Threshhold', 'States0.5', 'Threshhold0.5'], axis=1)
Relative_jobs_drop = ((States_sum_drop['sum_work_positions']-States_sum_drop['sum_work_positions0.5'])/States_sum_drop['sum_work_positions'])*100
States_sum_drop.head()

,States,sum_work_positions,Abbreviation,sum_work_positions0.5
4,California,17701210,CA,8055980
43,Texas,13326870,TX,5872910
32,New York,10149740,NY,4771540
9,Florida,9647870,FL,4122890
13,Illinois,6447580,IL,3008160


In [ ]:
Relative_jobs_drop_DF = pd.DataFrame({'Lost jobs ratio':Relative_jobs_drop.values})
Relative_jobs_drop_States = States_sum_drop.join(Relative_jobs_drop_DF)
Relative_jobs_drop_States_sort = Relative_jobs_drop_States.sort_values(by=['sum_work_positions'], ascending=False)
Relative_jobs_drop_mean = Relative_jobs_drop_States_sort['Lost jobs ratio'].mean()
Relative_jobs_drop_mean

54.966563375478984

In [ ]:
barplot(Relative_jobs_drop_States_sort, 'Lost jobs ratio', 'States', "Lost jobs ratio per state when we lost jobs with automatisation probability equal to 0.7 (Threshhold) or higher", None, Relative_jobs_drop_mean)

In [ ]:
Relative_jobs_drop_States_sort.head()

,States,sum_work_positions,Abbreviation,sum_work_positions0.5,Lost jobs ratio
4,California,17701210,CA,8055980,53.344356
43,Texas,13326870,TX,5872910,59.124660
32,New York,10149740,NY,4771540,61.456403
9,Florida,9647870,FL,4122890,54.386316
13,Illinois,6447580,IL,3008160,53.770951


In [ ]:
# Use Plotly Express to create a choropleth plot
fig = px.choropleth(Relative_jobs_drop_States_sort,
                    locations='Abbreviation',  # Column with state names
                    locationmode='USA-states',  # Use USA states as location mode
                    color='Lost jobs ratio',  # Column with values to be represented by color
                    scope='usa',  # Set the scope to USA
                    color_continuous_scale='jet',  # Choose a color scale
                    labels={'Lost jobs ratio': 'Job loss ratio'},  # Set custom labels
                    title='Lost job ratio per state with automation probability of 0.7+ threshold',
                    hover_name='States',  # Add state names to hover information
                    )    # Set a title for the plot

fig.update_geos(
    visible=False, resolution=110, scope="usa",
    showcountries=True, countrycolor="Black",
    showsubunits=True, subunitcolor="Blue"
)

fig.update_layout(geo=dict(bgcolor='white',  # Set background color to white
                            lakecolor='white'),  # Set lake color to white
                  margin=dict(r=20, l=20, t=50, b=20),  # Set margin for the plot
                  coloraxis_colorbar=dict(title='Job Loss Ratio',  # Set colorbar title
                                           ticksuffix='%'),  # Set colorbar tick suffix to percentage
                  title=dict(x=0.5, y=0.95),  # Set title position
                  )
# Show the plot
fig.show()


In [ ]:
Relative_jobs_drop_States_highest = Relative_jobs_drop_States.sort_values(by=['Lost jobs ratio'], ascending=False)

In [ ]:
barplot(Relative_jobs_drop_States_highest, 'Lost jobs ratio', 'States', "Lost jobs ratio per state when we lost jobs with automatisation probability equal to 0.7 (Threshhold) or higher", None, Relative_jobs_drop_mean)

Vermont and Texas have most jobs losses. Lets look, what are biggest occupations they lost

In [ ]:
Vermont = combined_data_2[['occupation', 'Probability', 'Texas', 'Vermont']].sort_values(by=['Probability'], ascending=False)
Vermont_full = Vermont.loc[(Vermont.Probability >= THERSHOLD)].sort_values(by=['Vermont'], ascending=False).reset_index()
Vermont = Vermont_full.head(9)
Vermont_tail = Vermont_full.tail(308).Vermont.sum()
df2 = {'occupation': 'Other', 'Probability': 0, 'Texas': 0, 'Vermont': Vermont_tail}
Vermont = Vermont.append(df2, ignore_index = True)
Vermont

<ipython-input-88-19227bc9de89>:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



,index,occupation,Probability,Texas,Vermont
0,395.0,cashiers,0.97,272270,9700
1,396.0,cashiers,0.97,272270,9700
2,400.0,retail salespersons,0.92,383080,9100
3,399.0,retail salespersons,0.92,383080,9100
4,485.0,"secretaries and administrative assistants, exc...",0.96,176670,5970
5,431.0,"bookkeeping, accounting, and auditing clerks",0.98,125140,5450
6,430.0,"bookkeeping, accounting, and auditing clerks",0.98,125140,5450
7,348.0,waiters and waitresses,0.94,219680,5400
8,347.0,waiters and waitresses,0.94,219680,5400
9,NaN,Other,0.00,0,35830


In [ ]:
import plotly.express as px

# Create a list of colors
colors = ["#E13F29", "#D69A80", "#D63B59", "#AE5552", "#CB5C3B", "#EB8076", "#96624E"]

def plot_pie(data_in, title_in, labels_in):
    '''
    Inputs:
        data_in: Data Frame of objects and floats;
        title_in: string, chart title;
        labels_in: object, occupation name.
    Output:
        pie chart.
    '''
    fig = px.pie(data_in,
                 values=data_in,
                 names=labels_in,
                 color_discrete_sequence=colors,  # Set colors
                 title=title_in,
                 hover_name=labels_in,
                 labels={'value': 'Percentage'},  # Set custom labels
                 hole=0.4,  # Set donut hole size
                 )
    # fig.update_traces(textposition='outside',  # Set text position inside the pie
    #                   textinfo='percent+label')  # Set text info to display percent and label
    fig.update_layout(title_text=title_in, title_font=dict(size=20),  # Set title font size and weight
                      showlegend=True)  # Hide legend

    fig.update_traces(pull=0.05)

    fig.show()  # Show the chart


In [ ]:
plot_pie(Vermont['Vermont'], "The largest most likely automatable occupations in Vermont", Vermont['occupation'])

In [ ]:
Texas = combined_data_2[['occupation', 'Probability', 'Texas']].sort_values(by=['Probability'], ascending=False)
Texas_full = Texas.loc[(Texas.Probability >= THERSHOLD)].sort_values(by=['Texas'], ascending=False).reset_index()
Texas = Texas_full.head(9)
Texas_tail = Texas_full.tail(308)['Texas'].sum()
df2 = {'occupation': 'Other', 'Probability': 0, 'Texas': Texas_tail}
Texas = Texas.append(df2, ignore_index = True)
Texas

<ipython-input-91-346d2098a086>:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



,index,occupation,Probability,Texas
0,400.0,retail salespersons,0.92,383080
1,399.0,retail salespersons,0.92,383080
2,495.0,"office clerks, general",0.96,363020
3,494.0,"office clerks, general",0.96,363020
4,396.0,cashiers,0.97,272270
5,395.0,cashiers,0.97,272270
6,348.0,waiters and waitresses,0.94,219680
7,347.0,waiters and waitresses,0.94,219680
8,788.0,"laborers and freight, stock, and material move...",0.85,178880
9,NaN,Other,0.00,1678860


In [ ]:
plot_pie(Texas['Texas'], "The largest most likely automatable occupations in Texas", Texas['occupation'])

District of Oregon has lowest sensitivity for automation. Let's check the data

In [ ]:
Oregon = combined_data_2[['occupation', 'Probability', 'Oregon']].sort_values(by=['Probability'], ascending=False)
Oregon_full = Oregon.loc[(Oregon.Probability >= THERSHOLD)].sort_values(by=['Oregon'], ascending=False).reset_index()
Oregon = Oregon_full.head(9)
Oregon_tail = Oregon_full.tail(308)['Oregon'].sum()
df2 = {'occupation': 'Other', 'Probability': 0, 'Oregon': Oregon_tail}
Oregon = Oregon.append(df2, ignore_index = True)
Oregon

<ipython-input-93-ff3520a4465d>:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



,index,occupation,Probability,Oregon
0,399.0,retail salespersons,0.92,61610
1,400.0,retail salespersons,0.92,61610
2,395.0,cashiers,0.97,45730
3,396.0,cashiers,0.97,45730
4,495.0,"office clerks, general",0.96,33500
5,494.0,"office clerks, general",0.96,33500
6,348.0,waiters and waitresses,0.94,33100
7,347.0,waiters and waitresses,0.94,33100
8,485.0,"secretaries and administrative assistants, exc...",0.96,26820
9,NaN,Other,0.00,252520


In [ ]:
plot_pie(Oregon['Oregon'], "The largest most likely automatable occupations in Oregon", Oregon['occupation'])